### **Post-Training Dynamic Quantization**

딥러닝 모델의 각 층(layer)의 Activation을 모델의 inference 중에 quantization시키는 것을 dynamic quantization이라고 부른다.

딥러닝의 입력 Tensor value에 따라 quantization parameter(clipping range)가 달라질 수 있고, 따라서 입력 텐서값에 따라 clippling range를 보정해주는 것이 dynamic quantization이다.

각 층의 activation후에 quantization layer를 추가하면 된다.

Activation 전에 quantization을 할 수도 있지만, 이 경우 quantization threshold가 증가하고, 따라서 quantization step이 증가되어 quantization loss가 커질 수 있다.



### **Import Packages**

In [ ]:
import os
#import pickle
#import time
#import datetime
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
#import torch.optim as optim

import torchvision.transforms as transforms
from torchvision import datasets

In [ ]:
#check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available. Training on CPU ...')

else:
    print('CUDA is available! Training on GPU ...')

### **Accuracy Functions**

In [ ]:
#output: model output
#target: correct answer
#topk: number of k best model output

def accuracy(output, target ,topk=(1,)):
  """
  Computes the accuracy over the k top predictions for the specified values of k
  IN top-5 accuracy you give yourself credit for having the right answer
  if the right answer appears in your top five guesses
  """
  with torch.no_grad(): #no need for gradient
    maxk = max(topk) #store biggest topk value
    batch_size=target.size(0) #check batch size // express "target" tensor's 0th dimension size

    _, pred = output.topk(maxk,1,True,True) #maxk=bring upper k value / dim=selected class dimension / 1st True=largest, if smallest use False / 2nd True=sort result? yes=True, no=False
    pred = pred.t() #transpose tensor pred

    #correct = pred.eq(target. view(1, -1).expand_as(pred))
    #correct = (pred == target.view(1, -1).expand_as(pred))
    correct = (pred == target.unsqueeze(dim=0)).expand_as(pred)

    res = []
    for k in topk:
      #correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
      correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
      res.append(correct_k.mul(1.0/batch_size))
    return res

class AverageMeter(object):
  """Computes and stores the average and current value"""

  def __init__(self, name, fmt=':f'): #floating point format
    self.name = name
    self.fmt = fmt
    self.reset()

  def reset(self):
    self.val=0
    self.avg=0
    self.sum=0
    self.count=0

  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.num / self.count

  def __str__(self):
    #fmtstr = '{name} {val' + self.fmt + '}({avg' + self.fmt+ '})'
    fmtstr = '{name} ({avg' + self.fmt +  '})'

    return fmtstr.format(**self.__dict__)

def norm1(X,Y):
  return np.sum(np.abs(X-Y))

def norm2(X,Y):
  return np.sqrt(np.sum(np.square(X-Y)))


### **Define Dynamic Quantized Model After Activation**

In [ ]:
class Darknet19q1(nn.Module):
    #quantized model after activation
    def __init__(self, num_classes: int = 1000):
        super(Darknet19q1, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm5 = nn.BatchNorm2d(128)

        self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm7 = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm8 = nn.BatchNorm2d(256)

        self.conv9 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm9 = nn.BatchNorm2d(512)
        self.conv10 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm10 = nn.BatchNorm2d(256)
        self.conv11 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm11 = nn.BatchNorm2d(512)
        self.conv12 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm12 = nn.BatchNorm2d(256)
        self.conv13 = nn.Conv2d(in_channels=256, out_channels=512,kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm13 = nn.BatchNorm2d(512)

        self.conv14 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm14 = nn.BatchNorm2d(1024)
        self.conv15 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm15 = nn.BatchNorm2d(512)
        self.conv16 = nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm16 = nn.BatchNorm2d(1024)
        self.conv17 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm17 = nn.BatchNorm2d(512)
        self.conv18 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm18 = nn.BatchNorm2d(1024)

        self.conv19 = nn.Conv2d(in_channels=1024, out_channels=num_classes, kernel_size=1, stride=1, padding=1, bias=False)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

    def quantize(self, X, NBIT=8):
        # 1. find threshold(maximum range)
        threshold = torch.maximum(X.max().abs(), X.min().abs())
        QRANGE = 2**(NBIT-1)
        # 2. find p: decimal position of quantized value
        p = torch.log2((QRANGE-1)/threshold).int()

        # 3. quantize using 8 bit
        # 3.1 apply thresholds
        data_th = torch.clamp(X, -QRANGE*torch.float_power(2, -p), (QRANGE-1)*torch.float_power(2,-p))

        # 3.2 calculate the scale factor for quantization
        SCALE = torch.float_power(2,-p)

        # 3.3 quantize(apply scale factor)
        #we are using a rounding function to force the quantized values to be the whole numbers which INT8 can represent
        data_qn = torch.round(data_th/SCALE)
        #torch.round(data_th/SCALE, deciamls=3)

        # 3.4 dequantize(simply the reverse quantization)
        data_dqn = data_qn*SCALE
        return data_dqn
    
    def forward(self, x):
        out = self.batchnorm1(self.conv1(x))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = F.max_pool2d(out, 2, stride=2)

        out = self.batchnorm2(self.conv2(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = F.max_pool2d(out, 2, stride=2)

        out = self.batchnorm3(self.conv3(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm4(self.conv4(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm5(self.conv5(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = F.max_pool2d(out, 2, stride=2)

        out = self.batchnorm6(self.conv6(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm7(self.conv7(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm8(self.conv8(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = F.max_pool2d(out, 2, stride=2)

        out = self.batchnorm9(self.conv9(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm10(self.conv10(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm11(self.conv11(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm12(self.conv12(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm13(self.conv13(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = F.max_pool2d(out, 2, stride=2)

        out = self.batchnorm14(self.conv14(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm15(self.conv15(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm16(self.conv16(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm17(self.conv17(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)
        out = self.batchnorm18(self.conv18(out))
        out = F.leaky_relu(out, negative_slope=0.1)
        out = self.quantize(out)

        out = self.conv19(out)
        out = self.quantize(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)

        return out


### **Build Model**

In [ ]:
model = Darknet19q1()
#print(model)

#weight_path = "C:\Bigdata\"quantization\darknet19_224d.pth"
#model.Load_state_dict(torch.Load(weigtht_path, map_Location=torch.device('cpu')))

weight_path = './darknet19_224d.pth'
model.load_state_dict(torch.load(weight_path))

### **Load Dataset**

In [ ]:
data_path = "\wrk\xsjhdnobkup5\taeheej\dataset\imagenet"
traindir = os.path.join(data_path, 'train')
valdir = os.path.join(data_path, 'val3k')

normalize = transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1,1,1]) #scale[0,1]--->normalization unpreceded

batch_size=32
num_workers=16

train_transform = transforms.Compose([
      transforms.RandomResizedCrop(224), #무작위로 이미지를 크롭(잘라내고)하고 224*224 pixel로 크기 조정
      transforms.RandomHorizontalFlip(), #이미지를 수평 방향으로 무작위로 뒤집기-데이터 다양성을 증가. 이미지 방향에 덜 민감
      transforms.ToTensor(), #이미지를 pytorch 텐서(pytorch 모델에서 처리할 수 있는 데이터 형식)로 변환. 이미지 데이터는 0에서 255 범위의 정수에서 0.0에서 1.0 범위의 부동소수점으로 스케일링
      normalize,
    ])

valid_transform=transforms.Compose([
      transforms.Resize(256), #모든 이미지의 크기를 256x256 픽셀로 조절한다.
      transforms.CenterCrop(224), #중앙을 기준으로 224x224로 크롭하여 크기 조정
      transforms.ToTensor(), #이미지를 pytorch 텐서로 변환
      normalize
    ])

train_dataset=datasets.ImageFolder(traindir, train_transform) #traindir 이미지를 가져와서 train_transform 전처리를 거치고 이미지 데이터셋을 생성한다.

valid_dataset=datasets.ImageFolder(valdir, valid_transform)

train_loader=torch.utils.data.DataLoader( #데이터 로드 및 처리
        train_dataset, #사용될 데이터셋
        batch_size=batch_size, #모델에 한번에 공급될 샘플수
        shuffle=True, #각 에프크(epoch) 시작시 데이터셋을 무작위로 섞어서 데이터의 순서에 의존하지 않도록 한다. overfitting을 방지한다.
        num_workers=num_workers,
        pin_memory=True) #Dataloader가 GPU에 복사하기 전에 CPU의 고정된 메모리 영역(pin된 메모리)에 로드하도록 한다. 이는 CPU에서 GPU로의 데이터 전송 속도를 향상시켜 학습 성능을 높인다.

valid_loader=torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True)

print("number of training dataset:%d" % len(train_dataset))
print("number of validation dataset:%d" % len(valid_dataset))

### **Accuracy of Dynamic Quantized Model**

In [ ]:
model.eval() #model.eval(): 모델을 평가 모드로 설정한다. 
             #이는 모델이 학습(training) 모드가 아닌, 평가(evaluation) 모드로 실행되어야 할 때 사용된다. 
             #평가 모드에서는 모델의 행동이 바뀌는 일부 레이어들(예를 들어, Dropout, BatchNorm 등)이 평가 상황에 적합하게 동작하도록 조정된다. 
             #예를 들어, Dropout 레이어는 학습 동안에는 일부 뉴런을 무작위로 비활성화하지만, 평가 모드에서는 모든 뉴런을 활성화 상태로 유지한다.

model.cuda() #model.cuda(): 모델의 매개변수와 버퍼를 CUDA를 사용 가능한 GPU 메모리로 이동시킵니다. 
             #이를 통해 모델이 GPU에서 실행될 수 있게 하며, GPU의 병렬 처리 능력을 활용하여 더 빠른 계산이 가능해집니다. 
             #이 메소드를 호출하기 전에는 모델이 CPU 메모리에 있었을 것이고, 이 호출을 통해 모델의 연산이 GPU에서 수행되도록 변경됩니다.

top1 = AverageMeter('Acc@1', ':6.4f') #batch 마다 평균을 구해서 그 정확도를 계산한다. 총 6자리를 표현하고 소수점 넷째자리까지 표시한다. Fixed point 형식으로 나타낸다
                                      #3.141592 --> 3.1416
top5 = AverageMeter('Acc@5', ':6.4f')
for data, target in valid_loader: #배치 단위로 data와 target을 순회 
    if train_on_gpu:
        data, target = data.cuda(), target.cuda() #data와 target을 GPU로 이동
    #forward pass: compute predicted outputs by passing inputs to the model
    output = model(data) #model에 data 전달
    acc1, acc5 = accuracy(output, target, topk=(1, 5)) #Top-1과 Top-5 정확도를 계산
    top1.update(acc1.item(), target.size(0)) #acc1.item()은 Top-1 정확도의 스칼라 값을 반환, target.size(0)는 현재 배치의 크기를 의미
    top5.update(acc5.item(), target.size(0)) 

##top1 accuracy, top5 accuracy
print(top1, top5)    